In [1]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.models import load_model

In [33]:
# Define your folder structure
data_dir = 'training_data'
classes = ['cat', 'dog']

# Load and preprocess audio data
def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []

    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr=None)
                # Perform preprocessing (e.g., convert to Mel spectrogram and resize)
                mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
                mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
                data.append(mel_spectrogram)
                labels.append(i)

    return np.array(data), np.array(labels)

# Split data into training and testing sets
data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))  # Convert labels to one-hot encoding
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Create a neural network model
input_shape = X_train[0].shape
input_layer = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(len(classes), activation='softmax')(x)
model = Model(input_layer, output_layer)

In [34]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
# Train the model
model.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/40
10/10 [==============================] - 4s 296ms/step - loss: 18.5190 - accuracy: 0.7517 - val_loss: 14.1559 - val_accuracy: 0.8108
Epoch 2/40
10/10 [==============================] - 3s 255ms/step - loss: 0.6512 - accuracy: 0.9048 - val_loss: 6.6608 - val_accuracy: 0.8108
Epoch 3/40
10/10 [==============================] - 3s 254ms/step - loss: 0.1621 - accuracy: 0.9694 - val_loss: 4.9493 - val_accuracy: 0.7973
Epoch 4/40
10/10 [==============================] - 3s 264ms/step - loss: 0.1314 - accuracy: 0.9728 - val_loss: 5.0750 - val_accuracy: 0.7838
Epoch 5/40
10/10 [==============================] - 3s 255ms/step - loss: 0.0993 - accuracy: 0.9728 - val_loss: 5.8756 - val_accuracy: 0.7838
Epoch 6/40
10/10 [==============================] - 3s 264ms/step - loss: 0.0640 - accuracy: 0.9728 - val_loss: 6.5862 - val_accuracy: 0.7838
Epoch 7/40
10/10 [==============================] - 3s 264ms/step - loss: 0.0452 - accuracy: 0.9796 - val_loss: 7.3807 - val_accuracy: 0.7703
Epoc

In [43]:
test_accuracy=model.evaluate(X_test,y_test,verbose=2)
print(test_accuracy[1])

3/3 - 0s - loss: 9.4607 - accuracy: 0.8108 - 134ms/epoch - 45ms/step
0.8108108043670654


In [45]:
# Save the model
model.save('audio_classification_model.hdf5')

In [1]:
# Load the saved model
model = load_model('audio_classification_model.hdf5')

# Define the target shape for input spectrograms
target_shape = (128, 128)

# Define your class labels
classes = ['cat', 'dog']

# Function to preprocess and classify an audio file
def test_audio(file_path, model):
    # Load and preprocess the audio file
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
    mel_spectrogram = tf.reshape(mel_spectrogram, (1,) + target_shape + (1,))

    # Make predictions
    predictions = model.predict(mel_spectrogram)

    # Get the class probabilities
    class_probabilities = predictions[0]

    # Get the predicted class index
    predicted_class_index = np.argmax(class_probabilities)

    return class_probabilities, predicted_class_index

# Test an audio file
test_audio_file = 'C:/Users/Admin/Desktop/training_data/Cat_effect.wav'
class_probabilities, predicted_class_index = test_audio(test_audio_file, model)

# Display results for all classes
for i, class_label in enumerate(classes):
    probability = class_probabilities[i]
    print(f'Class: {class_label}, Probability: {probability:.4f}')

# Calculate and display the predicted class and accuracy
predicted_class = classes[predicted_class_index]
accuracy = class_probabilities[predicted_class_index]
print(f'The audio is classified as: {predicted_class}')
print(f'Accuracy: {accuracy:.4f}')

NameError: name 'load_model' is not defined